In [62]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

In [63]:
## Load the trained model
model = load_model('model.h5')


## Load the encoders and scaler
with open('ohe_geography.pkl', 'rb') as file:
    ohe_geography = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [64]:
## Example input data - convert 'Geography' and 'Gender' to numerical
input_data = {
    'CreditScore': 300,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 240000,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': '1',
    'EstimatedSalary': 80000
}

input_data

{'CreditScore': 300,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 35,
 'Tenure': 5,
 'Balance': 240000,
 'NumOfProducts': 3,
 'HasCrCard': 1,
 'IsActiveMember': '1',
 'EstimatedSalary': 80000}

In [65]:
## One-hot encode 'Geography'
geo_encoded = ohe_geography.transform([[input_data['Geography']]]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geography.get_feature_names_out(['Geography']))

geo_encoded_df

/usr/local/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0

In [67]:
input_df = pd.DataFrame([input_data])

input_df

CreditScore Geography Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0          300    France   Male  ...          1               1            80000

[1 rows x 10 columns]

In [68]:
## Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [69]:
input_df

CreditScore Geography  Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0          300    France       1  ...          1               1            80000

[1 rows x 10 columns]

In [70]:
## Concatinate one-hot encoding
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [71]:
input_df

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0          300       1  ...                0.0              0.0

[1 rows x 12 columns]

In [72]:
## Scale the input data
input_scaled = scaler.transform(input_df)

In [73]:
input_scaled

array([[-3.64930447e+00,  9.13247552e-01, -3.70568589e-01,
        -1.34471949e-03,  2.62416692e+00,  2.53355998e+00,
         6.49202671e-01,  9.74816989e-01, -3.55232099e-01,
         1.00150113e+00, -5.79467227e-01, -5.76388018e-01]])

In [74]:
## Predict churn
prediction = model.predict(input_scaled)

prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


array([[0.9319353]], dtype=float32)

In [77]:
prediction_probability = prediction[0][0]

prediction_probability

0.9319353

In [78]:
## Predict whether customer stops using bank service (close their account)
if prediction_probability > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is likely to churn.
